# Part 5: 멀티모달 — 표(Table) 데이터 → Knowledge Graph

**소요시간**: 약 2시간  
**목표**: 표 형태의 구조화 데이터를 KG로 변환하고, 텍스트 KG와 통합하는 파이프라인 구축  
**도메인**: 제조 (브레이크패드 생산 공정)

---

## 학습 내용

| 섹션 | 내용 | 핵심 기술 |
|------|------|----------|
| 1 | 환경 설정 | OpenAI + Neo4j 연결 |
| 2 | OCR vs VLM 패러다임 비교 | 표 인식 기술 이해 |
| 3 | 표 데이터 → 구조화 | GPT-4o 트리플 변환 |
| 4 | 텍스트 + 표 통합 KG | MERGE 기반 멀티소스 통합 |
| 5 | 계층 구조 쿼리 | Document/Table 계층 연결 |
| 6 | 시각화 + 검증 | 통합 그래프 분석 |
| 7 | 연습 문제 | 실전 과제 |

---

## 1. 환경 설정

OpenAI API와 Neo4j 데이터베이스를 연결하고, 제조 도메인 샘플 데이터를 준비합니다.

In [ ]:
# 필수 패키지 설치 확인
# pip install openai neo4j python-dotenv pandas matplotlib

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from neo4j import GraphDatabase
import pandas as pd

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI 클라이언트
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Neo4j 드라이버
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASS = os.getenv("NEO4J_PASSWORD", "graphrag2024")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))

# 연결 테스트
with driver.session() as session:
    result = session.run("RETURN 1 AS test")
    print(f"Neo4j 연결 성공: {result.single()['test']}")

print(f"OpenAI API 키 설정 완료: {os.getenv('OPENAI_API_KEY')[:8]}...")

In [ ]:
# ============================================================
# 제조 도메인 샘플 데이터 정의
# - 브레이크패드 생산 공정의 표 형태 데이터 3종
# ============================================================

# (1) 공정 스펙표
process_spec_table = [
    {"공정명": "원재료 배합", "온도범위": "20~30°C", "압력": "상압", "소요시간": "30분", "사용장비": "MX-200 혼합기"},
    {"공정명": "프리폼 성형", "온도범위": "80~100°C", "압력": "50 MPa", "소요시간": "15분", "사용장비": "HP-01 유압프레스"},
    {"공정명": "열압착 성형", "온도범위": "150~180°C", "압력": "100 MPa", "소요시간": "20분", "사용장비": "HP-02 열압착기"},
    {"공정명": "열처리 경화", "온도범위": "200~250°C", "압력": "상압", "소요시간": "4시간", "사용장비": "HT-500 열처리로"},
    {"공정명": "연삭 가공", "온도범위": "상온", "압력": "N/A", "소요시간": "10분", "사용장비": "GR-100 연삭기"},
    {"공정명": "도장 코팅", "온도범위": "60~80°C", "압력": "상압", "소요시간": "30분", "사용장비": "CT-300 도장설비"},
]

# (2) 품질 검사 데이터
quality_inspection_table = [
    {"검사항목": "마찰계수", "기준값": "0.35~0.45", "측정값": "0.41", "판정": "합격"},
    {"검사항목": "경도 (HRR)", "기준값": "40~60", "측정값": "52", "판정": "합격"},
    {"검사항목": "두께 편차", "기준값": "±0.1mm", "측정값": "0.08mm", "판정": "합격"},
    {"검사항목": "접착 강도", "기준값": "≥15 MPa", "측정값": "12.3 MPa", "판정": "불합격"},
    {"검사항목": "내열 시험", "기준값": "300°C/2h", "측정값": "300°C/2h 통과", "판정": "합격"},
    {"검사항목": "소음 테스트", "기준값": "<70 dB", "측정값": "65 dB", "판정": "합격"},
]

# (3) 원재료 스펙
material_spec_table = [
    {"재료명": "페놀수지", "공급사": "한화솔루션", "인장강도": "60 MPa", "내열온도": "250°C"},
    {"재료명": "아라미드 섬유", "공급사": "코오롱인더스트리", "인장강도": "3,600 MPa", "내열온도": "400°C"},
    {"재료명": "마찰재 분말", "공급사": "상신브레이크", "인장강도": "N/A", "내열온도": "350°C"},
    {"재료명": "철판 (백플레이트)", "공급사": "포스코", "인장강도": "400 MPa", "내열온도": "800°C"},
    {"재료명": "접착제", "공급사": "헨켈코리아", "인장강도": "25 MPa", "내열온도": "200°C"},
]

# pandas DataFrame으로 확인
print("=== 공정 스펙표 ===")
df_process = pd.DataFrame(process_spec_table)
display(df_process)

print("\n=== 품질 검사 데이터 ===")
df_quality = pd.DataFrame(quality_inspection_table)
display(df_quality)

print("\n=== 원재료 스펙 ===")
df_material = pd.DataFrame(material_spec_table)
display(df_material)

---

## 2. OCR vs VLM 패러다임 비교

### 전통 OCR의 한계

기존 OCR(Optical Character Recognition)은 이미지에서 **문자만** 추출합니다.  
표의 구조(셀 병합, 행/열 관계)는 인식하지 못하고, 추출된 텍스트에서 맥락이 손실됩니다.

```
OCR 출력 예시:
"원재료 배합 20~30°C 상압 30분 MX-200 혼합기"
→ 어떤 값이 어떤 컬럼에 속하는지 알 수 없음!
```

### VLM (Vision Language Model)의 장점

VLM은 이미지를 **시각적으로 이해**합니다.  
표의 구조를 보존하고, 셀 병합을 처리하며, 맥락까지 파악합니다.

```
VLM 출력 예시:
{"공정명": "원재료 배합", "온도범위": "20~30°C", "압력": "상압", ...}
→ 구조화된 JSON으로 정확하게 추출!
```

### 비교표

| 기능 | OCR (전통) | VLM (GPT-4o 등) |
|------|-----------|------------------|
| 인식 대상 | 문자만 | 시각적 구조 전체 |
| 구조 보존 | 손실 | 보존 |
| 셀 병합 처리 | 불가 | 가능 |
| 맥락 이해 | 불가 | 가능 |
| 출력 형식 | 평문 텍스트 | JSON / 구조화 데이터 |
| 후처리 필요성 | 높음 (수동 파싱) | 낮음 (바로 사용 가능) |

> **핵심**: VLM은 표를 "이미지"가 아니라 "구조화된 데이터"로 이해합니다.  
> 이 노트북에서는 VLM 대신 **GPT-4o의 텍스트 기반 표 파싱**을 활용합니다.  
> (이미지가 아닌 Python dict로 표현된 표 데이터를 사용)

---

## 3. 표 데이터 → 구조화 (트리플 변환)

GPT-4o를 사용하여 표 형태의 데이터를 Knowledge Graph 트리플(주어-관계-목적어)로 변환합니다.

In [ ]:
# ============================================================
# GPT-4o로 표 데이터 → JSON 트리플 변환
# ============================================================

def table_to_triples(table_data: list[dict], table_name: str, domain_context: str) -> dict:
    """표 데이터를 GPT-4o로 KG 트리플(주어-관계-목적어)로 변환합니다."""
    
    prompt = f"""당신은 Knowledge Graph 전문가입니다.
다음 표 데이터를 분석하여 KG 트리플(주어-관계-목적어)로 변환하세요.

도메인 컨텍스트: {domain_context}
표 이름: {table_name}

표 데이터:
{json.dumps(table_data, ensure_ascii=False, indent=2)}

변환 규칙:
1. 각 행에서 엔티티(노드)를 식별하세요
2. 엔티티 간의 관계를 정의하세요
3. 속성값은 엔티티의 property로 설정하세요
4. 노드 라벨은 영문 PascalCase로 통일하세요 (예: Process, Equipment, Material)
5. 관계 타입은 영문 UPPER_SNAKE_CASE로 통일하세요 (예: USES_EQUIPMENT, HAS_SPEC)

출력 형식 (JSON):
{{
  "nodes": [
    {{"id": "고유ID", "label": "노드라벨", "properties": {{"name": "이름", ...}}}}
  ],
  "relationships": [
    {{"source": "시작노드ID", "target": "끝노드ID", "type": "관계타입", "properties": {{}}}}
  ]
}}

JSON만 반환하세요."""
    
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)

print("table_to_triples 함수 정의 완료")

In [ ]:
# ============================================================
# 3종 표 데이터 → 트리플 변환 실행
# ============================================================

domain_ctx = "자동차 브레이크패드 제조 공정. 원재료 배합부터 최종 검사까지의 생산 라인."

# (1) 공정 스펙표 변환
print("[1/3] 공정 스펙표 변환 중...")
process_triples = table_to_triples(
    process_spec_table, 
    "공정 스펙표", 
    domain_ctx
)
print(f"  - 노드: {len(process_triples.get('nodes', []))}개")
print(f"  - 관계: {len(process_triples.get('relationships', []))}개")

# (2) 품질 검사 데이터 변환
print("\n[2/3] 품질 검사 데이터 변환 중...")
quality_triples = table_to_triples(
    quality_inspection_table,
    "품질 검사 데이터",
    domain_ctx
)
print(f"  - 노드: {len(quality_triples.get('nodes', []))}개")
print(f"  - 관계: {len(quality_triples.get('relationships', []))}개")

# (3) 원재료 스펙 변환
print("\n[3/3] 원재료 스펙 변환 중...")
material_triples = table_to_triples(
    material_spec_table,
    "원재료 스펙",
    domain_ctx
)
print(f"  - 노드: {len(material_triples.get('nodes', []))}개")
print(f"  - 관계: {len(material_triples.get('relationships', []))}개")

print("\n=== 변환 완료 ===")

In [ ]:
# ============================================================
# 변환 결과 검증
# ============================================================

def validate_triples(triples: dict, table_name: str) -> None:
    """변환된 트리플의 품질을 검증합니다."""
    nodes = triples.get("nodes", [])
    rels = triples.get("relationships", [])
    
    print(f"\n=== {table_name} 검증 ===")
    
    # 1. 노드 라벨 분포
    labels = {}
    for node in nodes:
        label = node.get("label", "UNKNOWN")
        labels[label] = labels.get(label, 0) + 1
    print(f"노드 라벨 분포: {labels}")
    
    # 2. 관계 타입 분포
    rel_types = {}
    for rel in rels:
        rt = rel.get("type", "UNKNOWN")
        rel_types[rt] = rel_types.get(rt, 0) + 1
    print(f"관계 타입 분포: {rel_types}")
    
    # 3. 고립 노드 검사 (관계가 없는 노드)
    node_ids = {n["id"] for n in nodes}
    connected_ids = set()
    for rel in rels:
        connected_ids.add(rel["source"])
        connected_ids.add(rel["target"])
    orphan_ids = node_ids - connected_ids
    if orphan_ids:
        print(f"  [경고] 고립 노드 {len(orphan_ids)}개: {orphan_ids}")
    else:
        print(f"  [통과] 고립 노드 없음")
    
    # 4. 댕글링 관계 검사 (존재하지 않는 노드 참조)
    dangling = []
    for rel in rels:
        if rel["source"] not in node_ids:
            dangling.append(f"source={rel['source']}")
        if rel["target"] not in node_ids:
            dangling.append(f"target={rel['target']}")
    if dangling:
        print(f"  [경고] 댕글링 관계 {len(dangling)}개: {dangling[:5]}")
    else:
        print(f"  [통과] 댕글링 관계 없음")
    
    # 5. 샘플 출력
    print(f"\n  샘플 노드: {json.dumps(nodes[0], ensure_ascii=False) if nodes else 'N/A'}")
    print(f"  샘플 관계: {json.dumps(rels[0], ensure_ascii=False) if rels else 'N/A'}")


validate_triples(process_triples, "공정 스펙표")
validate_triples(quality_triples, "품질 검사 데이터")
validate_triples(material_triples, "원재료 스펙")

---

## 4. 텍스트 + 표 통합 KG 구축

Part 3에서 추출한 텍스트 기반 엔티티와 표에서 추출한 엔티티를 하나의 KG로 통합합니다.  
핵심은 **MERGE**: 동일 이름의 엔티티는 중복 생성하지 않고 병합합니다.

In [ ]:
# ============================================================
# 기존 그래프 초기화 (실습용)
# 주의: 기존 데이터가 있다면 삭제됩니다
# ============================================================

with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
    print("기존 그래프 초기화 완료")

In [ ]:
# ============================================================
# (A) 텍스트 문서에서 추출한 엔티티 시뮬레이션 (Part 3 방식)
# 실제로는 Part 3 노트북의 결과물을 사용합니다.
# 여기서는 학습을 위해 수동으로 생성합니다.
# ============================================================

text_entities_cypher = """
// 텍스트에서 추출한 공정 노드 (source: text)
MERGE (p1:Process {name: '원재료 배합'})
SET p1.source = ['text'], p1.description = '페놀수지, 아라미드 섬유, 마찰재 분말을 배합하는 공정'

MERGE (p2:Process {name: '프리폼 성형'})
SET p2.source = ['text'], p2.description = '배합된 재료를 금형에서 예비 성형하는 공정'

MERGE (p3:Process {name: '열압착 성형'})
SET p3.source = ['text'], p3.description = '프리폼을 백플레이트에 고온 압착하는 공정'

MERGE (p4:Process {name: '열처리 경화'})
SET p4.source = ['text'], p4.description = '성형된 패드를 열처리하여 경화하는 공정'

MERGE (p5:Process {name: '연삭 가공'})
SET p5.source = ['text'], p5.description = '경화된 패드를 규격에 맞게 연삭하는 공정'

MERGE (p6:Process {name: '도장 코팅'})
SET p6.source = ['text'], p6.description = '연삭된 패드에 방청 코팅을 하는 공정'

// 공정 순서 관계
MERGE (p1)-[:NEXT]->(p2)
MERGE (p2)-[:NEXT]->(p3)
MERGE (p3)-[:NEXT]->(p4)
MERGE (p4)-[:NEXT]->(p5)
MERGE (p5)-[:NEXT]->(p6)

// 텍스트에서 추출한 결함 노드
MERGE (d1:Defect {name: '접착 박리'})
SET d1.source = ['text'], d1.description = '마찰재와 백플레이트 사이의 접착이 분리되는 결함'

MERGE (d2:Defect {name: '크랙 발생'})
SET d2.source = ['text'], d2.description = '마찰재 표면에 균열이 발생하는 결함'

// 결함-공정 관계
MERGE (d1)-[:CAUSED_BY]->(p3)
MERGE (d2)-[:CAUSED_BY]->(p4)

// 텍스트에서 추출한 문서 노드
MERGE (doc1:Document {name: '브레이크패드 생산 매뉴얼'})
SET doc1.source = ['text'], doc1.type = 'manual'

MERGE (chunk1:Chunk {name: 'chunk_001'})
SET chunk1.text = '브레이크패드 제조는 원재료 배합부터 시작된다. 페놀수지와 아라미드 섬유를 정확한 비율로 혼합해야 한다.',
    chunk1.source = ['text']

MERGE (chunk2:Chunk {name: 'chunk_002'})
SET chunk2.text = '열압착 성형 시 온도와 압력이 부적절하면 접착 박리 결함이 발생할 수 있다. 150~180°C, 100MPa 범위를 유지해야 한다.',
    chunk2.source = ['text']

MERGE (doc1)-[:HAS_CHUNK]->(chunk1)
MERGE (doc1)-[:HAS_CHUNK]->(chunk2)
MERGE (chunk1)-[:MENTIONS]->(p1)
MERGE (chunk2)-[:MENTIONS]->(p3)
MERGE (chunk2)-[:MENTIONS]->(d1)
"""

with driver.session() as session:
    session.run(text_entities_cypher)
    print("텍스트 기반 엔티티 적재 완료 (source: text)")
    
    # 적재 확인
    result = session.run("MATCH (n) RETURN labels(n)[0] AS label, count(n) AS cnt ORDER BY cnt DESC")
    for record in result:
        print(f"  {record['label']}: {record['cnt']}개")

In [ ]:
# ============================================================
# (B) 표에서 추출한 트리플 → Neo4j 적재
# MERGE를 사용하여 동일 이름 엔티티는 병합
# source 속성으로 출처 추적
# ============================================================

def load_triples_to_neo4j(triples: dict, source_name: str):
    """변환된 트리플을 Neo4j에 적재합니다. MERGE로 중복 방지."""
    
    nodes = triples.get("nodes", [])
    rels = triples.get("relationships", [])
    
    with driver.session() as session:
        # 노드 적재
        for node in nodes:
            label = node.get("label", "Entity")
            props = node.get("properties", {})
            node_name = props.get("name", node.get("id", "unnamed"))
            
            # MERGE로 중복 방지 + source 속성 추가
            query = f"""
            MERGE (n:{label} {{name: $name}})
            SET n += $props
            SET n.source = CASE 
                WHEN n.source IS NULL THEN [$source]
                WHEN NOT $source IN n.source THEN n.source + $source
                ELSE n.source
            END
            """
            session.run(query, name=node_name, props=props, source=source_name)
        
        # 관계 적재
        for rel in rels:
            # source/target ID로 노드 이름 찾기
            src_node = next((n for n in nodes if n["id"] == rel["source"]), None)
            tgt_node = next((n for n in nodes if n["id"] == rel["target"]), None)
            
            if not src_node or not tgt_node:
                continue
            
            src_label = src_node.get("label", "Entity")
            tgt_label = tgt_node.get("label", "Entity")
            src_name = src_node.get("properties", {}).get("name", src_node["id"])
            tgt_name = tgt_node.get("properties", {}).get("name", tgt_node["id"])
            rel_type = rel.get("type", "RELATED_TO")
            rel_props = rel.get("properties", {})
            
            query = f"""
            MATCH (a:{src_label} {{name: $src_name}})
            MATCH (b:{tgt_label} {{name: $tgt_name}})
            MERGE (a)-[r:{rel_type}]->(b)
            SET r += $props
            SET r.source = $source
            """
            session.run(query, src_name=src_name, tgt_name=tgt_name, props=rel_props, source=source_name)
    
    print(f"  [{source_name}] 노드 {len(nodes)}개, 관계 {len(rels)}개 적재 완료")


# 3종 표 데이터 적재
print("표 데이터 → Neo4j 적재 시작")
print("(MERGE 사용 → 텍스트 엔티티와 동일 이름이면 병합됨)\n")

load_triples_to_neo4j(process_triples, "table_process_spec")
load_triples_to_neo4j(quality_triples, "table_quality_inspection")
load_triples_to_neo4j(material_triples, "table_material_spec")

print("\n=== 표 데이터 적재 완료 ===")

In [ ]:
# ============================================================
# 통합 확인: 텍스트 + 표 출처가 모두 있는 노드 확인
# ============================================================

with driver.session() as session:
    # source 속성이 여러 출처를 포함하는 노드 (= 통합된 노드)
    result = session.run("""
        MATCH (n)
        WHERE n.source IS NOT NULL AND size(n.source) > 1
        RETURN labels(n)[0] AS label, n.name AS name, n.source AS sources
        ORDER BY size(n.source) DESC
    """)
    
    merged_nodes = list(result)
    print(f"=== 멀티소스 통합 노드: {len(merged_nodes)}개 ===")
    for record in merged_nodes:
        print(f"  [{record['label']}] {record['name']} ← 출처: {record['sources']}")
    
    if not merged_nodes:
        print("  (통합된 노드가 없습니다. GPT-4o 변환 결과의 노드 이름이 다를 수 있습니다.)")
        print("  → 수동으로 MERGE 쿼리를 실행하여 통합할 수 있습니다.")

In [ ]:
# ============================================================
# 수동 통합 보강 (GPT-4o가 이름을 다르게 생성한 경우)
# 표의 엔티티를 텍스트 엔티티와 명시적으로 연결
# ============================================================

manual_merge_cypher = """
// 표의 장비 노드와 공정 노드 연결
MATCH (p:Process)
MATCH (e:Equipment)
WHERE p.name = e.used_in OR p.name CONTAINS e.process_name OR e.name CONTAINS p.name
MERGE (p)-[:USES_EQUIPMENT]->(e)
RETURN count(*) AS linked
"""

# 원재료 → 공정 연결
material_link_cypher = """
MATCH (m:Material)
MATCH (p:Process {name: '원재료 배합'})
MERGE (p)-[:USES_MATERIAL]->(m)
RETURN count(*) AS linked
"""

# 품질 검사 → 결함 연결
quality_link_cypher = """
MATCH (qi:QualityInspection)
WHERE qi.judgment = '불합격' OR qi.result = 'fail'
MATCH (d:Defect)
WHERE qi.name CONTAINS '접착' AND d.name CONTAINS '접착'
MERGE (qi)-[:INDICATES]->(d)
RETURN count(*) AS linked
"""

with driver.session() as session:
    print("수동 통합 보강 실행...")
    
    try:
        r1 = session.run(manual_merge_cypher).single()
        print(f"  장비-공정 연결: {r1['linked'] if r1 else 0}건")
    except Exception as e:
        print(f"  장비-공정 연결: 해당 노드 없음 (스킵) — {e}")
    
    try:
        r2 = session.run(material_link_cypher).single()
        print(f"  원재료-공정 연결: {r2['linked'] if r2 else 0}건")
    except Exception as e:
        print(f"  원재료-공정 연결: 해당 노드 없음 (스킵) — {e}")
    
    try:
        r3 = session.run(quality_link_cypher).single()
        print(f"  품질검사-결함 연결: {r3['linked'] if r3 else 0}건")
    except Exception as e:
        print(f"  품질검사-결함 연결: 해당 노드 없음 (스킵) — {e}")

    print("\n수동 통합 보강 완료")

---

## 5. 계층 구조 쿼리

텍스트와 표 데이터는 서로 다른 계층 구조를 가집니다.  
이 두 계층을 연결하면 **멀티소스 질의**가 가능합니다.

```
텍스트 계층:  Document → Chunk → Entity
표 계층:      Table → Row → Cell (= Property)
연결 지점:    Entity ← MERGE → 표에서 추출한 Node
```

In [ ]:
# ============================================================
# 표 계층 구조 노드 생성
# Table → TableRow → 기존 엔티티로 연결
# ============================================================

table_hierarchy_cypher = """
// Table 노드 생성
MERGE (t1:Table {name: '공정 스펙표'})
SET t1.source = ['table_process_spec'], t1.columns = ['공정명', '온도범위', '압력', '소요시간', '사용장비']

MERGE (t2:Table {name: '품질 검사 데이터'})
SET t2.source = ['table_quality_inspection'], t2.columns = ['검사항목', '기준값', '측정값', '판정']

MERGE (t3:Table {name: '원재료 스펙'})
SET t3.source = ['table_material_spec'], t3.columns = ['재료명', '공급사', '인장강도', '내열온도']

// 공정 스펙표의 Row 노드
MERGE (r1:TableRow {name: 'row_열압착성형'})
SET r1.공정명 = '열압착 성형', r1.온도범위 = '150~180°C', r1.압력 = '100 MPa', r1.소요시간 = '20분', r1.사용장비 = 'HP-02 열압착기'

MERGE (r2:TableRow {name: 'row_열처리경화'})
SET r2.공정명 = '열처리 경화', r2.온도범위 = '200~250°C', r2.압력 = '상압', r2.소요시간 = '4시간', r2.사용장비 = 'HT-500 열처리로'

// Table → Row 계층
MERGE (t1)-[:HAS_ROW]->(r1)
MERGE (t1)-[:HAS_ROW]->(r2)

// Row → 기존 Process 엔티티 연결
MATCH (p:Process {name: '열압착 성형'})
MERGE (r1)-[:DESCRIBES]->(p)

MATCH (p:Process {name: '열처리 경화'})
MERGE (r2)-[:DESCRIBES]->(p)

// Document → Table 관계 (같은 매뉴얼 소속)
MATCH (doc:Document {name: '브레이크패드 생산 매뉴얼'})
MERGE (doc)-[:CONTAINS_TABLE]->(t1)
MERGE (doc)-[:CONTAINS_TABLE]->(t2)
MERGE (doc)-[:CONTAINS_TABLE]->(t3)
"""

with driver.session() as session:
    session.run(table_hierarchy_cypher)
    print("표 계층 구조 생성 완료")
    print("  Document → Table → TableRow → Entity 계층 연결됨")

In [ ]:
# ============================================================
# 계층 구조 연결 쿼리 테스트
# ============================================================

queries = [
    {
        "title": "Document → Chunk 계층 (텍스트)",
        "cypher": """
            MATCH (doc:Document)-[:HAS_CHUNK]->(chunk:Chunk)-[:MENTIONS]->(entity)
            RETURN doc.name AS document, chunk.name AS chunk, labels(entity)[0] AS entity_type, entity.name AS entity_name
        """
    },
    {
        "title": "Table → Row 계층 (표)",
        "cypher": """
            MATCH (table:Table)-[:HAS_ROW]->(row:TableRow)-[:DESCRIBES]->(entity)
            RETURN table.name AS table_name, row.name AS row, labels(entity)[0] AS entity_type, entity.name AS entity_name
        """
    },
    {
        "title": "두 계층 연결: '이 공정의 스펙표에서 온도 범위는?'",
        "cypher": """
            MATCH (chunk:Chunk)-[:MENTIONS]->(p:Process)<-[:DESCRIBES]-(row:TableRow)<-[:HAS_ROW]-(table:Table)
            RETURN p.name AS 공정명, 
                   row.온도범위 AS 표_온도범위, 
                   row.압력 AS 표_압력,
                   chunk.text AS 텍스트_설명,
                   table.name AS 출처_표
        """
    },
    {
        "title": "멀티소스 질의: 접착 박리 결함의 원인 공정 스펙",
        "cypher": """
            MATCH (d:Defect {name: '접착 박리'})-[:CAUSED_BY]->(p:Process)
            OPTIONAL MATCH (p)<-[:DESCRIBES]-(row:TableRow)
            OPTIONAL MATCH (chunk:Chunk)-[:MENTIONS]->(p)
            RETURN d.name AS 결함, 
                   p.name AS 원인공정,
                   row.온도범위 AS 스펙_온도,
                   row.압력 AS 스펙_압력,
                   chunk.text AS 관련_텍스트
        """
    }
]

with driver.session() as session:
    for q in queries:
        print(f"\n=== {q['title']} ===")
        result = session.run(q["cypher"])
        records = list(result)
        if records:
            df = pd.DataFrame([dict(r) for r in records])
            display(df)
        else:
            print("  (결과 없음)")

---

## 6. 시각화 + 검증

통합된 그래프의 통계를 확인하고, 멀티홉 쿼리로 품질을 검증합니다.

In [ ]:
# ============================================================
# 통합 그래프 통계
# ============================================================

import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

with driver.session() as session:
    # 노드 라벨별 통계
    node_stats = session.run("""
        MATCH (n)
        RETURN labels(n)[0] AS label, count(n) AS count
        ORDER BY count DESC
    """)
    node_data = [(r["label"], r["count"]) for r in node_stats]
    
    # 관계 타입별 통계
    rel_stats = session.run("""
        MATCH ()-[r]->()
        RETURN type(r) AS rel_type, count(r) AS count
        ORDER BY count DESC
    """)
    rel_data = [(r["rel_type"], r["count"]) for r in rel_stats]
    
    # 전체 요약
    summary = session.run("""
        MATCH (n) WITH count(n) AS nodes
        MATCH ()-[r]->() WITH nodes, count(r) AS rels
        RETURN nodes, rels
    """).single()
    
    print(f"=== 통합 그래프 요약 ===")
    print(f"전체 노드: {summary['nodes']}개")
    print(f"전체 관계: {summary['rels']}개")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 노드 분포
if node_data:
    labels, counts = zip(*node_data)
    colors = plt.cm.Set3(range(len(labels)))
    axes[0].barh(labels, counts, color=colors)
    axes[0].set_title('노드 라벨별 분포', fontsize=14)
    axes[0].set_xlabel('개수')
    for i, v in enumerate(counts):
        axes[0].text(v + 0.1, i, str(v), va='center', fontweight='bold')

# 관계 분포
if rel_data:
    rel_labels, rel_counts = zip(*rel_data)
    colors = plt.cm.Pastel1(range(len(rel_labels)))
    axes[1].barh(rel_labels, rel_counts, color=colors)
    axes[1].set_title('관계 타입별 분포', fontsize=14)
    axes[1].set_xlabel('개수')
    for i, v in enumerate(rel_counts):
        axes[1].text(v + 0.1, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 소스별 노드 분포 (텍스트 vs 표)
# ============================================================

with driver.session() as session:
    source_stats = session.run("""
        MATCH (n)
        WHERE n.source IS NOT NULL
        UNWIND n.source AS src
        WITH CASE
            WHEN src = 'text' THEN 'text'
            WHEN src STARTS WITH 'table_' THEN 'table'
            ELSE 'other'
        END AS source_type, n
        RETURN source_type, count(DISTINCT n) AS node_count
        ORDER BY node_count DESC
    """)
    
    source_data = [(r["source_type"], r["node_count"]) for r in source_stats]
    
    print("=== 소스별 노드 분포 ===")
    for src, cnt in source_data:
        print(f"  {src}: {cnt}개")
    
    # 멀티소스 노드 (텍스트 + 표 모두 출처)
    multi_source = session.run("""
        MATCH (n)
        WHERE n.source IS NOT NULL AND size(n.source) > 1
        RETURN labels(n)[0] AS label, n.name AS name, n.source AS sources
    """)
    
    multi_records = list(multi_source)
    print(f"\n=== 멀티소스 노드 (텍스트+표 통합): {len(multi_records)}개 ===")
    for r in multi_records:
        print(f"  [{r['label']}] {r['name']} ← {r['sources']}")

In [ ]:
# ============================================================
# 멀티홉 쿼리 테스트
# ============================================================

multihop_tests = [
    {
        "question": "접착 박리 결함의 원인 공정에서 사용하는 장비의 스펙은? (3-hop)",
        "cypher": """
            MATCH (d:Defect {name: '접착 박리'})-[:CAUSED_BY]->(p:Process)
            OPTIONAL MATCH (p)-[:USES_EQUIPMENT]->(e:Equipment)
            OPTIONAL MATCH (p)<-[:DESCRIBES]-(row:TableRow)
            RETURN d.name AS defect, p.name AS process, 
                   e.name AS equipment,
                   row.온도범위 AS temp_range,
                   row.압력 AS pressure
        """
    },
    {
        "question": "원재료 배합 공정에 사용되는 재료의 내열온도는? (2-hop)",
        "cypher": """
            MATCH (p:Process {name: '원재료 배합'})-[:USES_MATERIAL]->(m:Material)
            RETURN p.name AS process, m.name AS material, 
                   m.heat_resistance AS heat_resistance
        """
    },
    {
        "question": "전체 공정 흐름과 각 공정의 스펙 요약 (2-hop)",
        "cypher": """
            MATCH path = (p1:Process)-[:NEXT*]->(pN:Process)
            WITH nodes(path) AS processes
            UNWIND processes AS p
            OPTIONAL MATCH (p)<-[:DESCRIBES]-(row:TableRow)
            RETURN p.name AS process, 
                   row.온도범위 AS temp_range,
                   row.소요시간 AS duration
            ORDER BY p.name
        """
    }
]

with driver.session() as session:
    for test in multihop_tests:
        print(f"\nQ: {test['question']}")
        print("-" * 60)
        try:
            result = session.run(test["cypher"])
            records = list(result)
            if records:
                df = pd.DataFrame([dict(r) for r in records])
                display(df)
            else:
                print("  (결과 없음 — 그래프 구조에 따라 다를 수 있습니다)")
        except Exception as e:
            print(f"  오류: {e}")

---

## 7. 연습 문제

### 연습 1: 새 표 데이터 추가

아래 **설비 점검 이력** 표를 KG에 추가하세요.

In [ ]:
# ============================================================
# 연습 1: 새 표 데이터 추가
# 설비 점검 이력 데이터를 트리플로 변환하고 Neo4j에 적재하세요
# ============================================================

# 설비 점검 이력 표
maintenance_table = [
    {"장비명": "HP-02 열압착기", "점검일": "2024-01-15", "점검항목": "온도 센서 교정", "결과": "정상", "다음점검": "2024-04-15"},
    {"장비명": "HP-02 열압착기", "점검일": "2024-04-15", "점검항목": "유압 실린더 점검", "결과": "교체 필요", "다음점검": "2024-04-20"},
    {"장비명": "HT-500 열처리로", "점검일": "2024-02-01", "점검항목": "히터 소자 점검", "결과": "정상", "다음점검": "2024-05-01"},
    {"장비명": "MX-200 혼합기", "점검일": "2024-03-10", "점검항목": "블레이드 마모도", "결과": "마모 진행중", "다음점검": "2024-06-10"},
]

print("설비 점검 이력:")
display(pd.DataFrame(maintenance_table))

# TODO: table_to_triples 함수를 사용하여 트리플로 변환하세요
# maintenance_triples = table_to_triples(maintenance_table, "설비 점검 이력", domain_ctx)

# TODO: load_triples_to_neo4j 함수를 사용하여 Neo4j에 적재하세요
# load_triples_to_neo4j(maintenance_triples, "table_maintenance")

# TODO: 기존 Equipment/Process 노드와 연결하세요

In [ ]:
# ============================================================
# 연습 2: 크로스 소스 쿼리 작성
# 아래 질문에 대한 Cypher 쿼리를 작성하세요
# ============================================================

# Q1: "유압 실린더 교체가 필요한 장비가 사용되는 공정의 결함 이력은?"
# 경로: MaintenanceRecord → Equipment → Process → Defect

# TODO: Cypher 쿼리를 작성하세요
q1_cypher = """
// 여기에 쿼리를 작성하세요
// MATCH (mr:MaintenanceRecord)-[...]->...
"""

# Q2: "접착 박리 결함을 예방하기 위해 점검해야 할 장비와 점검 항목은?"
# 경로: Defect → Process → Equipment → MaintenanceRecord

# TODO: Cypher 쿼리를 작성하세요
q2_cypher = """
// 여기에 쿼리를 작성하세요
// MATCH (d:Defect {name: '접착 박리'})-[...]->...
"""

# Q3: "내열온도가 300°C 이하인 원재료가 관련된 공정의 품질 검사 결과는?"
# 경로: Material → Process → QualityInspection

# TODO: Cypher 쿼리를 작성하세요
q3_cypher = """
// 여기에 쿼리를 작성하세요
"""

print("연습 2: 위 TODO 부분에 Cypher 쿼리를 작성하고 실행해 보세요.")
print("힌트: MATCH 패턴에서 노드 라벨과 관계 타입을 확인하세요.")
print("")
print("현재 그래프의 노드/관계 확인:")

with driver.session() as session:
    schema = session.run("""
        CALL db.schema.visualization()
    """)
    print("  Neo4j Browser에서 'CALL db.schema.visualization()' 으로 스키마를 확인하세요.")

In [ ]:
# ============================================================
# 정리: Neo4j 드라이버 종료
# ============================================================

driver.close()
print("Neo4j 드라이버 종료 완료")
print("")
print("=== Part 5 완료 ===")
print("")
print("학습 요약:")
print("  1. OCR vs VLM 패러다임 차이를 이해했습니다")
print("  2. GPT-4o로 표 데이터를 KG 트리플로 변환했습니다")
print("  3. MERGE를 사용하여 텍스트 KG와 표 KG를 통합했습니다")
print("  4. Document/Table 계층 구조를 연결했습니다")
print("  5. 멀티소스 질의로 텍스트+표를 넘나드는 검색을 수행했습니다")
print("")
print("다음 단계: Part 6 - Text2Cypher + 하이브리드 검색 파이프라인")